In [1]:
import numpy as np
from keras.applications import vgg16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Ladataan CIFAR-10 datasetti, jaetaan validaatiojoukkoon, normalisoidaan kuvat ja muunnetaan etiketit kategoriamuotoon.

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_val = x_train[:5000]
y_val = y_train[:5000]
x_train = x_train[5000:]
y_train = y_train[5000:]

x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

Ladataan VGG-16 ilman dense-luokittelijakerroksia ja asetetaan mallin mainot olemaan oppimattomia, koska ei haluta muuttaa esikoulutetuja konvoluutioalustan painoja nopeuden vuoksi ja säilyttääksemme mallin tarkkuuden. Tämän avulla voimme käyttää VGG-16:ta feature extractorina, eli saadaan tiivistetyt piirteet jokaisesta kuvasta.

In [3]:
conv_base = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False

def extract_features(data):
    return conv_base.predict(data)

x_train_features = extract_features(x_train)
x_val_features = extract_features(x_val)
x_test_features = extract_features(x_test)

313/313 [==============================] - 19s 61ms/step


Rakennetaan yksinkertainen dense-pohjainen luokittelija esikäsiteltyjen ominaisuuksien perusteella käyttäen kahta dense-kerrosta, jossa ensimmäisessä on aktivaatiofunktiona ReLU ja lopussa softmax-luokittelukerros ja kompiloidaan se käyttäen Adam-optimointia ja kategorista ristientropiaa.

In [4]:
malli = models.Sequential([
    layers.Input(shape=x_train_features.shape[1:]),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

malli.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

malli.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 133,898
Trainable params: 133,898
Non-trainable params: 0
_________________________________________________________________


In [5]:
history = malli.fit(x_train_features, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val_features, y_val))

Epoch 1/10
1407/1407 [==============================] - 3s 2ms/step - loss: 1.4325 - accuracy: 0.4975 - val_loss: 1.2483 - val_accuracy: 0.5692
Epoch 2/10
1407/1407 [==============================] - 2s 2ms/step - loss: 1.2407 - accuracy: 0.5665 - val_loss: 1.1729 - val_accuracy: 0.5870
Epoch 3/10
1407/1407 [==============================] - 2s 1ms/step - loss: 1.1803 - accuracy: 0.5876 - val_loss: 1.1473 - val_accuracy: 0.5994
Epoch 4/10
1407/1407 [==============================] - 2s 1ms/step - loss: 1.1439 - accuracy: 0.5999 - val_loss: 1.1348 - val_accuracy: 0.6000
Epoch 5/10
1407/1407 [==============================] - 2s 1ms/step - loss: 1.1102 - accuracy: 0.6102 - val_loss: 1.1158 - val_accuracy: 0.6138
Epoch 6/10
1407/1407 [==============================] - 2s 1ms/step - loss: 1.0841 - accuracy: 0.6205 - val_loss: 1.1063 - val_accuracy: 0.6088
Epoch 7/10
1407/1407 [==============================] - 2s 1ms/step - loss: 1.0637 - accuracy: 0.6259 - val_loss: 1.0939 - val_accuracy:

In [7]:
test_loss, test_acc = malli.evaluate(x_test_features, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 0s 850us/step - loss: 1.1105 - accuracy: 0.6104
Test accuracy: 0.6104000210762024


In [ ]:
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

# plt.title('Model Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

: 